In [109]:
from tensorflow.keras import layers, models, Input

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

data_directory = 'sketch_small'
TARGET_SIZE = (64, 64, 3)
EPOCHS = 10
BATCH_SIZE = 32

In [102]:
def create_pairs(directory):
    #directory is the path of the dataset (main folder)
    #folders are classes in dataset
    pairs = []
    labels = []
    
    folders = os.listdir(directory)
    for folder_name in folders:
        folder_path = os.path.join(directory, folder_name)
        
        if os.path.isdir(folder_path):
            images = os.listdir(folder_path)
            
            folder_length = len(images)
            for i in range(folder_length):
                for j in range(folder_length):
                    if i != j:
                        image_path = os.path.join(folder_path, images[i])
                        pairs.append([image_path, os.path.join(folder_path, images[j])])
                        labels.append(1)#positive pairs
                        
                        dif_folder = random.choice([x for x in folders if x != folder_name])
                        dif_folder_path = os.path.join(directory, dif_folder)
                        dif_image_path = os.path.join(dif_folder_path, random.choice(os.listdir(dif_folder_path)))

                        pairs.append([image_path, dif_image_path])
                        labels.append(0)#negative pairs
                        
    return np.array(pairs), np.array(labels) 

In [103]:
def process_images(pairs, target_size):
    images = []
     
    for pair in pairs:
      
        img1 = np.array(load_img(pair[0], target_size=target_size))
        img2 = np.array(load_img(pair[1], target_size=target_size))

        images.append((img1, img2))
        
    return images

In [104]:
def prepare_data(directory):
    X, y = create_pairs(directory)
    X = process_images(X, TARGET_SIZE)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

X_train, X_test, y_train, y_test = prepare_data(data_directory)  

In [105]:
X_train_left = X_train[:, 0]
X_train_right = X_train[:, 1]

In [106]:
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.models import Model

def initialize_base_branch():
    input = Input(shape=TARGET_SIZE, name="base_input")
    x = Flatten(name="flatten_input")(input)
    x = Dense(128, activation='relu', name="first_base_dense")(x)
    x = Dropout(0.3, name="first_dropout")(x)
    x = Dense(128, activation='relu', name="second_base_dense")(x)
    x = Dropout(0.3, name="second_dropout")(x)
    x = Dense(128, activation='relu', name="third_base_dense")(x)

    #Returning a Model, with input and outputs, not just a group of layers. 
    return Model(inputs=input, outputs=x)


In [107]:
def initialize_siamese_network(input_shape=TARGET_SIZE):
    input_left = Input(shape=input_shape)
    input_right = Input(shape=input_shape)
    
    base_branch = initialize_base_branch()
    
    encoded_left = base_branch(input_left)
    encoded_right = base_branch(input_right)
    
    L1_distance = Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])
    
    prediction = Dense(1, activation='sigmoid')(L1_distance)
    
    siamese_network = Model(inputs=[input_left, input_right], outputs=prediction)
    
    return siamese_network

siamese_model = initialize_siamese_network()


In [110]:
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.fit([X_train_left, X_train_right], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)


Epoch 1/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.5061 - loss: 0.6931 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 2/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.4984 - loss: 0.6932 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 3/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.5020 - loss: 0.6932 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 4/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.4935 - loss: 0.6932 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 5/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.4983 - loss: 0.6932 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 6/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.5059 - loss: 0.6931 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 7/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.5018 - loss: 0.6932 - val_accuracy: 0.4969 - val_loss: 0.6932
Epoch 8/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.4937 - loss: 0.6932 - val_accu